In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset('joelniklaus/brazilian_court_decisions')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['process_number', 'orgao_julgador', 'publish_date', 'judge_relator', 'ementa_text', 'decision_description', 'judgment_text', 'judgment_label', 'unanimity_text', 'unanimity_label'],
        num_rows: 3234
    })
    validation: Dataset({
        features: ['process_number', 'orgao_julgador', 'publish_date', 'judge_relator', 'ementa_text', 'decision_description', 'judgment_text', 'judgment_label', 'unanimity_text', 'unanimity_label'],
        num_rows: 404
    })
    test: Dataset({
        features: ['process_number', 'orgao_julgador', 'publish_date', 'judge_relator', 'ementa_text', 'decision_description', 'judgment_text', 'judgment_label', 'unanimity_text', 'unanimity_label'],
        num_rows: 405
    })
})

In [5]:
dataset['train'][0]

{'process_number': '0800304-08.2018.8.02.0000',
 'orgao_julgador': 'Tribunal Pleno',
 'publish_date': '12/03/2019',
 'judge_relator': 'Des. João Luiz Azevedo Lessa',
 'ementa_text': 'DIREITO PENAL E PROCESSUAL PENAL. REVISÃO CRIMINAL. ART. 621 DO CÓDIGO DE PROCESSO PENAL. REQUERENTE CONDENADO EM JÚRI POPULAR PELA PRÁTICA DOS CRIMES DE HOMICÍDIO DUPLAMENTE QUALIFICADO E HOMICÍDIO QUALIFICADO TENTADO. PLEITO DE REFAZIMENTO DA DOSIMETRIA DA PENA IMPOSTA AO REQUERENTE. ADMISSIBILIDADE NA VIA REVISIONAL. PRECEDENTES. ALEGAÇÃO DE ERRO NO PROCESSO DE DOSIMETRIA DA PENA. COMPORTAMENTO DA VÍTIMA. CIRCUNSTÂNCIA JUDICIAL NEUTRA QUE NÃO PODE SER CONSIDERADA DE FORMA DESFAVORÁVEL AO SENTENCIANDO SEGUNDO PRECEDENTES DO SUPERIOR TRIBUNAL DE JUSTIÇA E NOVO ENTENDIMENTO DA CÂMARA CRIMINAL DESTE TRIBUNAL DE JUSTIÇA. AFASTAMENTO. CULPABILIDADE. AUSÊNCIA DE EXPOSIÇÃO DE MOTIVOS PARA O INCREMENTO DA PENA-BASE. AFASTADO O DESVALOR. VALORAÇÃO ATRIBUÍDA ÀS CIRCUNSTÂNCIAS DO CRIME MANTIDA. FUNDAMENTAÇÃO IDÔNEA

In [6]:
train_texts = dataset['train']['decision_description']
train_labels = dataset['train']['judgment_label']

test_texts = dataset['test']['decision_description']
test_labels = dataset['test']['judgment_label']

print(f'\nTrain size: {len(train_texts)} -- {len(train_labels)}')
print(f'Test size: {len(test_texts)} -- {len(test_labels)}')


Train size: 3234 -- 3234
Test size: 405 -- 405


In [7]:
from collections import Counter

print(f'Train Labels Distribution: {Counter(train_labels)}')
print(f'Test Labels Distribution: {Counter(test_labels)}')

Train Labels Distribution: Counter({'no': 1960, 'partial': 677, 'yes': 597})
Test Labels Distribution: Counter({'no': 234, 'partial': 93, 'yes': 78})


In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(train_labels)

train_labels = label_encoder.transform(train_labels)
test_labels = label_encoder.transform(test_labels)

print(f'Train Labels Distribution: {Counter(train_labels)}')
print(f'Test Labels Distribution: {Counter(test_labels)}')

Train Labels Distribution: Counter({0: 1960, 1: 677, 2: 597})
Test Labels Distribution: Counter({0: 234, 1: 93, 2: 78})


In [9]:
!python -m spacy download pt_core_news_sm

2023-12-13 14:18:18.667460: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 14:18:18.667696: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 14:18:18.667753: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 14:18:18.685302: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 14:18:21.777132: W tensorflow/c

In [10]:
import spacy

from tqdm import tqdm

def preprocess_texts(list_texts):
  nlp = spacy.load('pt_core_news_sm', disable=['ner'])
  new_texts = []
  with tqdm(total=len(list_texts), desc='Preprocessing') as pbar:
    for text in list_texts:
      doc = nlp(text)
      tokens = [t.lemma_.lower() for t in doc if t.pos_ != 'PUNCT' and not t.is_stop]
      texto_normalizado = ' '.join(tokens)
      new_texts.append(texto_normalizado)
      pbar.update(1)
  return new_texts

In [11]:
train_texts = preprocess_texts(train_texts)

Preprocessing: 100%|██████████| 3234/3234 [01:03<00:00, 50.98it/s]


In [12]:
test_texts = preprocess_texts(test_texts)

Preprocessing: 100%|██████████| 405/405 [00:05<00:00, 70.68it/s]


In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# vectorizer_option = 'binary'
vectorizer_option = 'count'
# vectorizer_option = 'tf_idf'

vectorizer = None

if vectorizer_option == 'binary':
    vectorizer = CountVectorizer(binary=True, max_features=None, ngram_range=(1, 1))
elif vectorizer_option == 'count':
    vectorizer = CountVectorizer(binary=False, max_features=None, ngram_range=(1, 1))
elif vectorizer_option == 'tf_idf':
    vectorizer = TfidfVectorizer(max_features=None, ngram_range=(1, 1))

print(f'Vectorizer Option: {vectorizer_option}')


Vectorizer Option: count


In [14]:
X_train = vectorizer.fit_transform(train_texts).toarray()
X_test = vectorizer.transform(test_texts).toarray()

print(f'\nExample Raw Text: {train_texts[0]}')
print(f'\nExample Vectorized Text: {X_train[0]}')


Example Raw Text: direito penal processual penal revisão criminal artigo 621 código processo penal requerente condenado júri popular prática crimes homicídio duplamente qualificado homicídio qualificado tentado pleito refazimento dosimetria pena imposta requerente admissibilidade via revisional precedentes alegação erro processo dosimetria pena comportamento vítima circunstância judicial neutra considerada desfavorável sentenciando precedentes superior tribunal justiça entendimento câmara criminal tribunal justiça afastamento culpabilidade ausência exposição motivos incremento pena-base afastado desvalor valoração atribuída circunstâncias crime mantida fundamentação idônea pena-base reduzida compensação agravante motivação torpe atenuante confissão espontânea pena privativa liberdade redimensionada crime tentado aplicada fração redutora máxima ante distância atos praticados requerente consumação crime pena redimensionada

Example Vectorized Text: [0 0 0 ... 0 0 0]


In [15]:
print(f'Vocabulary: {len(vectorizer.vocabulary_)}')

Vocabulary: 12447


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import PassiveAggressiveClassifier
from xgboost import XGBClassifier

classifiers = {
    'Logistic Regression': LogisticRegression(class_weight='balanced', max_iter=500),
    'Multinomial NB': MultinomialNB(),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Quadratic Discriminant': QuadraticDiscriminantAnalysis(),
    'Passive Aggressive': PassiveAggressiveClassifier(),
    'MLP Classifier': MLPClassifier()
}


In [17]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

results_df = pd.DataFrame(columns=['Classificador', 'Acurácia', 'Precisão', 'Recall', 'F1-Score'])

for classifier_name, classifier in classifiers.items():
    print(f'\nClassifier: {classifier_name}')

    classifier.fit(X_train, train_labels)

    y_pred = classifier.predict(X_test)

    report = classification_report(test_labels, y_pred, output_dict=True, digits=5)

    print(classification_report(test_labels, y_pred))

    # ConfusionMatrixDisplay.from_estimator(classifier, X_test, test_labels, display_labels=['Negative', 'Positive']).plot()

    # plt.show()

    accuracy = accuracy_score(test_labels, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(test_labels, y_pred, average='weighted')

    results_df = results_df.append({
        'Classificador': classifier_name,
        'Acurácia': accuracy,
        'Precisão': precision,
        'Recall': recall,
        'F1-Score': f1
    }, ignore_index=True)

best_accuracy = results_df.loc[results_df['Acurácia'].idxmax(), 'Classificador']
results_df['Acurácia'] = results_df.apply(lambda x: f'**{x["Acurácia"] * 100:.3f}**' if x['Classificador'] == best_accuracy else f'{x["Acurácia"] * 100:.3f}', axis=1)

best_precision = results_df.loc[results_df['Precisão'].idxmax(), 'Classificador']
results_df['Precisão'] = results_df.apply(lambda x: f'**{x["Precisão"] * 100:.3f}**' if x['Classificador'] == best_precision else f'{x["Precisão"] * 100:.3f}', axis=1)

best_recall = results_df.loc[results_df['Recall'].idxmax(), 'Classificador']
results_df['Recall'] = results_df.apply(lambda x: f'**{x["Recall"] * 100:.3f}**' if x['Classificador'] == best_recall else f'{x["Recall"] * 100:.3f}', axis=1)

best_f1 = results_df.loc[results_df['F1-Score'].idxmax(), 'Classificador']
results_df['F1-Score'] = results_df.apply(lambda x: f'**{x["F1-Score"] * 100:.3f}**' if x['Classificador'] == best_f1 else f'{x["F1-Score"] * 100:.3f}', axis=1)

results_df.to_excel(f'resultados_classificadores_{vectorizer_option}.xlsx', index=False)

print(results_df)



Classifier: Logistic Regression
              precision    recall  f1-score   support

           0       0.83      0.82      0.82       234
           1       0.59      0.62      0.61        93
           2       0.64      0.63      0.63        78

    accuracy                           0.74       405
   macro avg       0.69      0.69      0.69       405
weighted avg       0.74      0.74      0.74       405


Classifier: Multinomial NB


<ipython-input-17-bda36663167d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


              precision    recall  f1-score   support

           0       0.78      0.60      0.68       234
           1       0.46      0.61      0.53        93
           2       0.45      0.59      0.51        78

    accuracy                           0.60       405
   macro avg       0.56      0.60      0.57       405
weighted avg       0.64      0.60      0.61       405


Classifier: KNN


<ipython-input-17-bda36663167d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


              precision    recall  f1-score   support

           0       0.67      0.86      0.76       234
           1       0.61      0.38      0.47        93
           2       0.43      0.26      0.32        78

    accuracy                           0.63       405
   macro avg       0.57      0.50      0.51       405
weighted avg       0.61      0.63      0.61       405


Classifier: SVM


...

              precision    recall  f1-score   support

           0       0.80      0.85      0.82       234
           1       0.63      0.63      0.63        93
           2       0.66      0.53      0.59        78

    accuracy                           0.74       405
   macro avg       0.70      0.67      0.68       405
weighted avg       0.73      0.74      0.73       405


Classifier: MLP Classifier


<ipython-input-17-bda36663167d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


              precision    recall  f1-score   support

           0       0.80      0.88      0.84       234
           1       0.68      0.63      0.66        93
           2       0.63      0.50      0.56        78

    accuracy                           0.75       405
   macro avg       0.70      0.67      0.68       405
weighted avg       0.74      0.75      0.74       405



<ipython-input-17-bda36663167d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


            Classificador    Acurácia    Precisão      Recall    F1-Score
0     Logistic Regression      73.580      73.827      73.580      73.690
1          Multinomial NB      60.000      64.430      60.000      61.078
2                     KNN      63.457      61.070      63.457      60.509
3                     SVM      69.136      70.608      69.136      65.777
4           Random Forest      72.840      75.101      72.840      69.903
5                 XGBoost  **77.531**  **77.102**  **77.531**  **76.665**
6           Decision Tree      68.395      68.255      68.395      68.321
7  Quadratic Discriminant      28.395      60.281      28.395      26.765
8      Passive Aggressive      73.827      73.325      73.827      73.380
9          MLP Classifier      74.815      73.955      74.815      74.128


In [18]:
# results_df.to_excel(f'resultados_classificadores_count.xlsx', index=False)